In [15]:
%%writefile mapper2.py

import sys
import re

def read_stopwords(file_path):
    return set(word.strip() for word in open(file_path))

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode
stop_words = read_stopwords('stop_words_en.txt')

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)
        if word.lower() in stop_words:
            print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % 1

Overwriting mapper2.py


In [16]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [17]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [18]:
%%writefile counter2.py

from __future__ import division
import sys

stop_words=0
total_words=1
for line in sys.stdin:
    try:
        key, val = line.strip().split('=', 1)
    except ValueError as e:
        continue
    if key == 'Stop words':
        stop_words = float(val)
    if key == 'Total words':
        total_words = float(val)

print ("%.3f" % (stop_words / total_words * 100))

Overwriting counter2.py


In [19]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.partition.keypartitioner.options=-k1,1 \
    -files mapper2.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper2.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> output.log

cat output.log | python ./counter2.py
cat output.log >&2

41.603


rm: `wordcount_result_1530213664603677': No such file or directory
18/06/28 19:21:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/06/28 19:21:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/06/28 19:21:11 INFO mapred.FileInputFormat: Total input files to process : 1
18/06/28 19:21:11 INFO mapreduce.JobSubmitter: number of splits:2
18/06/28 19:21:12 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1530192984314_0011
18/06/28 19:21:12 INFO impl.YarnClientImpl: Submitted application application_1530192984314_0011
18/06/28 19:21:12 INFO mapreduce.Job: The url to track the job: http://d8449248cc2a:8088/proxy/application_1530192984314_0011/
18/06/28 19:21:12 INFO mapreduce.Job: Running job: job_1530192984314_0011
18/06/28 19:21:22 INFO mapreduce.Job: Job job_1530192984314_0011 running in uber mode : false
18/06/28 19:21:22 INFO mapreduce.Job:  map 0% reduce 0%
18/06/28 19:21:51 INFO mapreduce.Job:  map 2% reduce 0%
18/06/28 19:21